In [0]:
import requests
from bs4 import BeautifulSoup
import json
from math import sin, cos, sqrt, atan2, radians
import operator
from IPython.display import HTML, display
from ipywidgets import interact
from geopy.geocoders import Nominatim

# Function used to find thde distance (miles) between two sets of GPS coordinates
def getDistance(userLat, userLon, lat, lon): #Source: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
  R = 6373.0
  lat1 = radians(userLat)
  lon1 = radians(userLon)
  lat2 = radians(lat)
  lon2 = radians(lon)

  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))
  distance = R * c / 1.60934
  return distance

# Main function called by interact
def getNearbyStations(Location):

  # Acquiring weather data for Chicago
  weatherURL = 'https://www.google.com/search?q=current+weather+chicago&rlz=1C1KMZB_enUS571US586&oq=current+weather+chicago&aqs=chrome..69i57j0l7.3494j1j7&sourceid=chrome&ie=UTF-8'
  weatherData = BeautifulSoup(requests.get(weatherURL).text, 'lxml')
  currentWeather = weatherData.find_all('div', class_='BNeawe tAd8D AP7Wnd')
  print(currentWeather[1].text)

  # Acquiring the station data for DivvyBike
  divvyBikeURL = 'https://gbfs.divvybikes.com/gbfs/en/station_information.json'
  divvyBikeData = json.loads(requests.get(divvyBikeURL).text)
  # Parsing to the individual stations
  stations = divvyBikeData["data"]["stations"]

  loc = Nominatim(user_agent="DivvyBike Weather Checker").geocode(Location) # Nominatim is a tool that takes an address and returns information about it
  if loc == None:
    print('\n')
    display(HTML('<b>Please input your location</b>'))
  else:
    userLatitude = loc.latitude
    userLongitude = loc.longitude

    # Creates a dictionary of ALL station's name and distances from current location
    # Sorts the dictionary and converts into a list tuples, in order to be later accessed by index
    stationDistances = {}
    stationCoordinates = {}

    for station in stations:
      stationName = station["name"]
      distance = getDistance(userLatitude, userLongitude, station["lat"], station["lon"])
      stationDistances[stationName] = distance
      stationCoordinates[stationName] = (station["lat"], station["lon"])

    sortedStationDistances = sorted(stationDistances.items(), key=operator.itemgetter(1)) # Stored as a list of tuples

    # Accesses the first 5 nearest station tuples and prints their information in readable format
    # Format for Google Maps directions HTML: '''<iframe src="https://www.google.com/maps/embed?pb=!1m20!1m8!1m3!1d1873.8740220889792!2d-87.61874409332205!3d41.885299471292534!3m2!1i1024!2i768!4f13.1!4m9!3e2!4m3!3m2!
    #                                          1d[initialLat]!2d[initialLon]!4m3!3m2!1d[finalLat]!2d[finalLon]!5e1!3m2!1sen!2sus!4v1586646251966!5m2!1sen!2sus"
    #                                          width="600" height="450" frameborder="0" style="border:0;" allowfullscreen="" aria-hidden="false" tabindex="0"></iframe>'''

    print('\n')
    for i in range(5): 
      stationTuple = sortedStationDistances[i]
      stationName = stationTuple[0]
      stationDistance = stationTuple[1]
      stationLat = stationCoordinates[stationName][0]
      stationLon = stationCoordinates[stationName][1]
      # stationURL = 'https://www.google.com/maps/dir/' + str(userLatitude) + ',' + str(userLongitude) + '/' + str(stationLat) + ',' + str(stationLon) + '/data=!3m1!1e3!4m2!4m1!3e2'
      mapString = '<iframe src="https://www.google.com/maps/embed?pb=!1m24!1m12!1m3!1d3747.7759008558023!2d-87.6209819845587!3d41.884824579221664!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!4m9!3e2!4m3!3m2!1d' + str(userLatitude) + '!2d' + str(userLongitude) + '!4m3!3m2!1d' + str(stationLat) + '!2d' + str(stationLon) + '!5e1!3m2!1sen!2sus!4v1586650602192!5m2!1sen!2sus" width="400" height="300" frameborder="0" style="border:0;" allowfullscreen="" aria-hidden="false" tabindex="0"></iframe>'
      display(HTML('<b>' + stationName + '</b>'))
      #print(str(i+1) + ': ' + stationName)
      print('Distance: ', '{:.2f}'.format(stationDistance), 'miles')
      display(HTML(mapString))
      print('\n')

interact(getNearbyStations, Location='')